# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [63]:
import pandas as pd
import requests
# Carregar a base de dados
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Contar o número de registros
num_registros = sinasc.shape[0]

# Contar o número de registros não duplicados
num_registros_unicos = sinasc.drop_duplicates().shape[0]

print(f"Número de registros: {num_registros}")
print(f"Número de registros não duplicados: {num_registros_unicos}")


Número de registros: 27028
Número de registros não duplicados: 27028


In [65]:
# Contando valores missing por variável
missing_por_variavel = sinasc.isnull().sum()
print(missing_por_variavel)


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [67]:
# criando a seleção de colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_selecionado = sinasc[colunas_interesse]

# Contar valores missing novamente
missing_por_variavel_selecionado = sinasc_selecionado.isnull().sum()
print(missing_por_variavel_selecionado)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [69]:
# Remover registros com Apgar5 não preenchido
sinasc_selecionado = sinasc_selecionado.dropna(subset=['APGAR5'])

# Contar número de linhas e valores missing após remoção
num_registros_pos_remocao = sinasc_selecionado.shape[0]
missing_pos_remocao = sinasc_selecionado.isnull().sum()

print(f"Número de registros após remoção de Apgar5 não preenchido: {num_registros_pos_remocao}")
print(missing_pos_remocao)


Número de registros após remoção de Apgar5 não preenchido: 26925
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [71]:
# Substituir valores faltantes de QTDFILVIVO por zero
sinasc_selecionado = sinasc_selecionado.copy()  # garantir que estamos trabalhando com uma cópia segura
sinasc_selecionado['QTDFILVIVO'] = sinasc_selecionado['QTDFILVIVO'].fillna(0)


In [73]:
# Substituir valores faltantes de 'ESTCIVMAE' e 'CONSULTAS' por 9
sinasc_selecionado['ESTCIVMAE'] = sinasc_selecionado['ESTCIVMAE'].fillna(9)
sinasc_selecionado['CONSULTAS'] = sinasc_selecionado['CONSULTAS'].fillna(9)


In [75]:
# Categorizar Apgar5
def categoriza_apgar5(valor):
    if 8 <= valor <= 10:
        return 'normal'
    elif 6 <= valor <= 7:
        return 'asfixia leve'
    elif 4 <= valor <= 5:
        return 'asfixia moderada'
    elif 0 <= valor <= 3:
        return 'asfixia severa'

sinasc_selecionado['APGAR5_categoria'] = sinasc_selecionado['APGAR5'].apply(categoriza_apgar5)

# Calcular frequências
frequencias_apgar5 = sinasc_selecionado['APGAR5_categoria'].value_counts()
print(frequencias_apgar5)


APGAR5_categoria
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64


In [77]:
# Renomear variáveis para snake case
sinasc_selecionado.columns = sinasc_selecionado.columns.str.lower()
sinasc_selecionado.rename(columns={
    'locnasc': 'loc_nasc', 'idademae': 'idade_mae', 'estcivmae': 'est_civ_mae',
    'escmae': 'esc_mae', 'qtdfilvivo': 'qtd_fil_vivo', 'gestacao': 'gestacao',
    'gravidez': 'gravidez', 'consultas': 'consultas', 'apgar5': 'apgar5'}, inplace=True)

print(sinasc_selecionado.head())



   loc_nasc  idade_mae  est_civ_mae          esc_mae  qtd_fil_vivo  \
0         1         19          5.0      8 a 11 anos           0.0   
1         1         29          2.0      8 a 11 anos           1.0   
2         1         37          9.0      8 a 11 anos           2.0   
3         1         30          5.0  12 anos ou mais           0.0   
4         1         30          2.0      8 a 11 anos           1.0   

          gestacao gravidez  consultas  apgar5 apgar5_categoria  
0  37 a 41 semanas    Única          4    10.0           normal  
1  37 a 41 semanas    Única          4     9.0           normal  
2  37 a 41 semanas    Única          4    10.0           normal  
3  37 a 41 semanas    Única          3    10.0           normal  
4  37 a 41 semanas    Única          4    10.0           normal  
